In [ ]:
!pip install --upgrade transformers

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import numpy as np

# Load your dataset
df = pd.read_csv("final_training_dataset.csv")

# Define post text and multi-labels
texts = df['post'].astype(str).tolist()

labels = df.iloc[:, -8:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
labels = (labels > 0).astype(int)


# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels.values, test_size=0.2, random_state=42
)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Custom dataset
class PostDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = PostDataset(train_encodings, train_labels)
test_dataset = PostDataset(test_encodings, test_labels)

# Load model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=train_labels.shape[1],
    problem_type="multi_label_classification"
)

# Training arguments
# TrainingArguments without evaluation_strategy and save_strategy
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
)



# Metric computation
def compute_metrics(pred):
    preds = torch.sigmoid(torch.tensor(pred.predictions)).numpy()
    preds_binary = (preds > 0.5).astype(int)
    return {
        "classification_report": classification_report(test_labels, preds_binary, output_dict=True)
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train!
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: as29624041 (as29624041-university-of-engineering-and-technology-taxila) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.670800
20,0.623400
30,0.614600
40,0.603000
50,0.592700
60,0.555100
70,0.584600
80,0.539900
90,0.521900
100,0.519400


TrainOutput(global_step=1692, training_loss=0.3433717763311192, metrics={'train_runtime': 467.9767, 'train_samples_per_second': 28.89, 'train_steps_per_second': 3.616, 'total_flos': 889363275939840.0, 'train_loss': 0.3433717763311192, 'epoch': 4.0})

In [4]:
# After training finishes
model.save_pretrained("/contents/saved_bert_model")
tokenizer.save_pretrained("/contents/saved_bert_model")

# # Save the trained model and tokenizer
# model_dir = "./saved_bert_model"
# trainer.save_model(model_dir)
# tokenizer.save_pretrained(model_dir)

('/contents/saved_bert_model/tokenizer_config.json',
 '/contents/saved_bert_model/special_tokens_map.json',
 '/contents/saved_bert_model/vocab.txt',
 '/contents/saved_bert_model/added_tokens.json')

In [5]:
model_dir = "/content/saved_bert_model"
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

('/content/saved_bert_model/tokenizer_config.json',
 '/content/saved_bert_model/special_tokens_map.json',
 '/content/saved_bert_model/vocab.txt',
 '/content/saved_bert_model/added_tokens.json')

In [4]:
# Zip the model folder
!zip -r saved_bert_model.zip /content/saved_bert_model

# Download to local system
from google.colab import files
files.download("saved_bert_model.zip")

  adding: content/saved_bert_model/ (stored 0%)
  adding: content/saved_bert_model/config.json (deflated 56%)
  adding: content/saved_bert_model/tokenizer_config.json (deflated 75%)
  adding: content/saved_bert_model/training_args.bin (deflated 52%)
  adding: content/saved_bert_model/model.safetensors (deflated 7%)
  adding: content/saved_bert_model/vocab.txt (deflated 53%)
  adding: content/saved_bert_model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Path to the saved model directory
model_dir = "/content/saved_bert_model"

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)
model.eval()  # Set model to evaluation mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
def predict(text, model, tokenizer, threshold=0.5):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits)

    # Apply threshold to get binary predictions
    preds = (probs > threshold).int().squeeze().tolist()
    return preds


In [7]:
text = "I am so tirified by training such models"
predictions = predict(text, model, tokenizer)

print("Predicted labels:", predictions)


Predicted labels: [1, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
label_names = ['toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'sexual_explicit', 'severe_toxic', 'spam', 'hate']

predicted_labels = [label_names[i] for i, val in enumerate(predictions) if val == 1]
print("Predicted categories:", predicted_labels)


Predicted categories: ['obscene', 'identity_hate', 'severe_toxic', 'hate']


In [8]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report

# 1. Load your test dataset
df_test = pd.read_csv('/content/final_validating_dataset.csv')

# 2. Prepare test inputs
texts = df_test['post'].astype(str).tolist()
true_labels = df_test.iloc[:, -8:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values

# 3. Load saved model and tokenizer
model_dir = '/content/saved_bert_model'  # Change if saved elsewhere
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)
model.eval()

# 4. Predict
predictions = []

with torch.no_grad():
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits)
        pred = (probs > 0.5).int().squeeze().tolist()
        predictions.append(pred)

# 5. Classification report
print(classification_report(true_labels, predictions, zero_division=0))


              precision    recall  f1-score   support

           0       0.73      0.72      0.72       346
           1       0.51      0.43      0.47       164
           2       0.74      0.76      0.75       358
           3       0.82      0.89      0.85       640
           4       0.80      0.77      0.78       453
           5       0.85      0.95      0.90       708
           6       0.70      0.72      0.71       224
           7       0.74      0.80      0.77       462

   micro avg       0.78      0.81      0.79      3355
   macro avg       0.74      0.75      0.74      3355
weighted avg       0.77      0.81      0.79      3355
 samples avg       0.76      0.80      0.74      3355



In [ ]:
import zipfile

zip_path = "/content/saved_bert_model.zip"  # or wherever the file is located
extract_path = "/content/saved_bert_model"  # output directory

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully to:", extract_path)


Unzipped successfully to: /content/saved_bert_model
